In [1]:
import pandas as pd
#pd.set_option('display.max_rows', None)
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import path
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy
import numpy as np
import netCDF4 as nc
np.set_printoptions(threshold=100000)
from shapely.geometry import Polygon, Point, MultiPoint
from shapely.ops import cascaded_union
from datetime import datetime, timedelta
import time
import warnings
import datetime
import math
from scipy.ndimage.interpolation import shift
import shapely.wkt
from shapely.validation import explain_validity
from shapely.validation import make_valid
import xarray as xr

warnings.filterwarnings('ignore')
import seaborn as sns


from joblib import Parallel, delayed
import multiprocessing

import subprocess as sp


/data2/lthapa/miniconda3/envs/ML_py/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [5]:
gridmet = xr.open_dataset('/data2/lthapa/2019/GRIDMET/gridmet_all_2019-06-01.nc')
print(gridmet)
print(gridmet['potential_evapotranspiration_alfalfa'].values[0:10,0:10])

<xarray.Dataset>
Dimensions:                                    (crs: 1, lat: 432, lon: 571)
Coordinates:
  * lon                                        (lon) float64 -124.8 ... -101.0
  * lat                                        (lat) float64 48.98 ... 31.02
    day                                        datetime64[ns] ...
  * crs                                        (crs) uint16 3
Data variables: (12/18)
    burning_index_g                            (lat, lon) float32 ...
    energy_release_component-g                 (lat, lon) float32 ...
    potential_evapotranspiration_alfalfa       (lat, lon) float32 ...
    dead_fuel_moisture_1000hr                  (lat, lon) float32 ...
    dead_fuel_moisture_100hr                   (lat, lon) float32 ...
    potential_evapotranspiration_grass         (lat, lon) float32 ...
    ...                                         ...
    min_air_temperature                        (lat, lon) float32 ...
    max_air_temperature                     

# Pre-process gridmet, including overnight lows

In [7]:
years = [2019,2020,2021]
#years = [2019]

for jj in range(len(years)):
    gridmet_filenames =os.popen('ls /data2/lthapa/'+str(years[jj])+'/GRIDMET/*_'+str(years[jj])+'.nc').read().split('\n')
    print(gridmet_filenames[0:len(gridmet_filenames)-1])
    dat_gridmet = xr.open_mfdataset(gridmet_filenames[0:len(gridmet_filenames)-1], combine='nested',compat='override', coords='all')

    days = np.arange(np.datetime64(str(years[jj])+'-06-01 00:00'),np.datetime64(str(years[jj]+1)+'-01-01 00:00'),np.timedelta64(1,'D'))

    for kk in range(len(days)):
        today = days[kk]
        print(today)
        #select today and make it western US
        dat_gridmet_today = dat_gridmet.sel(day=today,lon=slice(-125, -101), lat=slice(49,31))
        #print(dat_gridmet_today)
        
        #load in the 90th percentile for this day
        day_string = str(days[kk])[5:10]
        heatwave_high_threshold_today = xr.open_dataset('/data2/lthapa/TMAX_CLIMO/climo_tmmx_90th_15day_'+day_string+'.nc')
        heatwave_low_threshold_today = xr.open_dataset('/data2/lthapa/TMAX_CLIMO/climo_tmmn_90th_15day_'+day_string+'.nc')

        #print(heatwave_threshold_today)
        is_high_heatwave = xr.where(dat_gridmet_today['max_air_temperature']>heatwave_high_threshold_today['air_temperature'],
                              1,0)
        is_highlow_heatwave = xr.where((dat_gridmet_today['max_air_temperature']>heatwave_high_threshold_today['air_temperature'])&
                                       (dat_gridmet_today['min_air_temperature']>heatwave_low_threshold_today['air_temperature']),
                              1,0)

        dat_gridmet_today['is_high_heatwave'] = (('lat','lon'), is_high_heatwave)
        dat_gridmet_today['is_highlow_heatwave'] = (('lat','lon'), is_highlow_heatwave)
        print( dat_gridmet_today['is_high_heatwave'])
        
        #send to netcdf
        file_name_save = '/data2/lthapa/'+str(years[jj])+'/GRIDMET/gridmet_all_'+np.datetime_as_string(today,unit='D')+'.nc'
        print(file_name_save)
        dat_gridmet_today.to_netcdf(file_name_save)
        print()

        

['/data2/lthapa/2019/GRIDMET/bi_2019.nc', '/data2/lthapa/2019/GRIDMET/erc_2019.nc', '/data2/lthapa/2019/GRIDMET/etr_2019.nc', '/data2/lthapa/2019/GRIDMET/fm1000_2019.nc', '/data2/lthapa/2019/GRIDMET/fm100_2019.nc', '/data2/lthapa/2019/GRIDMET/pet_2019.nc', '/data2/lthapa/2019/GRIDMET/pr_2019.nc', '/data2/lthapa/2019/GRIDMET/rmax_2019.nc', '/data2/lthapa/2019/GRIDMET/rmin_2019.nc', '/data2/lthapa/2019/GRIDMET/sph_2019.nc', '/data2/lthapa/2019/GRIDMET/srad_2019.nc', '/data2/lthapa/2019/GRIDMET/th_2019.nc', '/data2/lthapa/2019/GRIDMET/tmmn_2019.nc', '/data2/lthapa/2019/GRIDMET/tmmx_2019.nc', '/data2/lthapa/2019/GRIDMET/vpd_2019.nc', '/data2/lthapa/2019/GRIDMET/vs_2019.nc']
2019-06-01T00:00
<xarray.DataArray 'is_high_heatwave' (lat: 432, lon: 571)>
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])
Coordinates:
  * lon      (lon) float64 


2019-06-15T00:00
<xarray.DataArray 'is_high_heatwave' (lat: 432, lon: 571)>
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])
Coordinates:
  * lon      (lon) float64 -124.8 -124.7 -124.7 -124.6 ... -101.1 -101.1 -101.0
  * lat      (lat) float64 48.98 48.94 48.9 48.86 ... 31.15 31.11 31.07 31.02
    day      datetime64[ns] 2019-06-15
/data2/lthapa/2019/GRIDMET/gridmet_all_2019-06-15.nc

2019-06-16T00:00
<xarray.DataArray 'is_high_heatwave' (lat: 432, lon: 571)>
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])
Coordinates:
  * lon      (lon) float64 -124.8 -124.7 -124.7 -124.6 ... -101.1 -101.1 -101.0
  * lat      (lat) float64 48.98 48.94 48.9 48.86 ... 31.15 31.11 31.07 31.02
  

<xarray.DataArray 'is_high_heatwave' (lat: 432, lon: 571)>
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])
Coordinates:
  * lon      (lon) float64 -124.8 -124.7 -124.7 -124.6 ... -101.1 -101.1 -101.0
  * lat      (lat) float64 48.98 48.94 48.9 48.86 ... 31.15 31.11 31.07 31.02
    day      datetime64[ns] 2019-06-30
/data2/lthapa/2019/GRIDMET/gridmet_all_2019-06-30.nc

2019-07-01T00:00
<xarray.DataArray 'is_high_heatwave' (lat: 432, lon: 571)>
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])
Coordinates:
  * lon      (lon) float64 -124.8 -124.7 -124.7 -124.6 ... -101.1 -101.1 -101.0
  * lat      (lat) float64 48.98 48.94 48.9 48.86 ... 31.15 31.11 31.07 31.02
    day      datetim


2019-07-16T00:00
<xarray.DataArray 'is_high_heatwave' (lat: 432, lon: 571)>
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])
Coordinates:
  * lon      (lon) float64 -124.8 -124.7 -124.7 -124.6 ... -101.1 -101.1 -101.0
  * lat      (lat) float64 48.98 48.94 48.9 48.86 ... 31.15 31.11 31.07 31.02
    day      datetime64[ns] 2019-07-16
/data2/lthapa/2019/GRIDMET/gridmet_all_2019-07-16.nc

2019-07-17T00:00
<xarray.DataArray 'is_high_heatwave' (lat: 432, lon: 571)>
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])
Coordinates:
  * lon      (lon) float64 -124.8 -124.7 -124.7 -124.6 ... -101.1 -101.1 -101.0
  * lat      (lat) float64 48.98 48.94 48.9 48.86 ... 31.15 31.11 31.07 31.02
  

<xarray.DataArray 'is_high_heatwave' (lat: 432, lon: 571)>
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])
Coordinates:
  * lon      (lon) float64 -124.8 -124.7 -124.7 -124.6 ... -101.1 -101.1 -101.0
  * lat      (lat) float64 48.98 48.94 48.9 48.86 ... 31.15 31.11 31.07 31.02
    day      datetime64[ns] 2019-07-31
/data2/lthapa/2019/GRIDMET/gridmet_all_2019-07-31.nc

2019-08-01T00:00
<xarray.DataArray 'is_high_heatwave' (lat: 432, lon: 571)>
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1]])
Coordinates:
  * lon      (lon) float64 -124.8 -124.7 -124.7 -124.6 ... -101.1 -101.1 -101.0
  * lat      (lat) float64 48.98 48.94 48.9 48.86 ... 31.15 31.11 31.07 31.02
    day      datetim

...

<xarray.DataArray 'is_high_heatwave' (lat: 432, lon: 571)>
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])
Coordinates:
  * lon      (lon) float64 -124.8 -124.7 -124.7 -124.6 ... -101.1 -101.1 -101.0
  * lat      (lat) float64 48.98 48.94 48.9 48.86 ... 31.15 31.11 31.07 31.02
    day      datetime64[ns] 2021-10-20
/data2/lthapa/2021/GRIDMET/gridmet_all_2021-10-20.nc

2021-10-21T00:00
<xarray.DataArray 'is_high_heatwave' (lat: 432, lon: 571)>
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])
Coordinates:
  * lon      (lon) float64 -124.8 -124.7 -124.7 -124.6 ... -101.1 -101.1 -101.0
  * lat      (lat) float64 48.98 48.94 48.9 48.86 ... 31.15 31.11 31.07 31.02
    day      datetim


2021-11-05T00:00
<xarray.DataArray 'is_high_heatwave' (lat: 432, lon: 571)>
array([[0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])
Coordinates:
  * lon      (lon) float64 -124.8 -124.7 -124.7 -124.6 ... -101.1 -101.1 -101.0
  * lat      (lat) float64 48.98 48.94 48.9 48.86 ... 31.15 31.11 31.07 31.02
    day      datetime64[ns] 2021-11-05
/data2/lthapa/2021/GRIDMET/gridmet_all_2021-11-05.nc

2021-11-06T00:00
<xarray.DataArray 'is_high_heatwave' (lat: 432, lon: 571)>
array([[0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])
Coordinates:
  * lon      (lon) float64 -124.8 -124.7 -124.7 -124.6 ... -101.1 -101.1 -101.0
  * lat      (lat) float64 48.98 48.94 48.9 48.86 ... 31.15 31.11 31.07 31.02
  

<xarray.DataArray 'is_high_heatwave' (lat: 432, lon: 571)>
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 0, 1]])
Coordinates:
  * lon      (lon) float64 -124.8 -124.7 -124.7 -124.6 ... -101.1 -101.1 -101.0
  * lat      (lat) float64 48.98 48.94 48.9 48.86 ... 31.15 31.11 31.07 31.02
    day      datetime64[ns] 2021-11-20
/data2/lthapa/2021/GRIDMET/gridmet_all_2021-11-20.nc

2021-11-21T00:00
<xarray.DataArray 'is_high_heatwave' (lat: 432, lon: 571)>
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])
Coordinates:
  * lon      (lon) float64 -124.8 -124.7 -124.7 -124.6 ... -101.1 -101.1 -101.0
  * lat      (lat) float64 48.98 48.94 48.9 48.86 ... 31.15 31.11 31.07 31.02
    day      datetim


2021-12-06T00:00
<xarray.DataArray 'is_high_heatwave' (lat: 432, lon: 571)>
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])
Coordinates:
  * lon      (lon) float64 -124.8 -124.7 -124.7 -124.6 ... -101.1 -101.1 -101.0
  * lat      (lat) float64 48.98 48.94 48.9 48.86 ... 31.15 31.11 31.07 31.02
    day      datetime64[ns] 2021-12-06
/data2/lthapa/2021/GRIDMET/gridmet_all_2021-12-06.nc

2021-12-07T00:00
<xarray.DataArray 'is_high_heatwave' (lat: 432, lon: 571)>
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])
Coordinates:
  * lon      (lon) float64 -124.8 -124.7 -124.7 -124.6 ... -101.1 -101.1 -101.0
  * lat      (lat) float64 48.98 48.94 48.9 48.86 ... 31.15 31.11 31.07 31.02
  

<xarray.DataArray 'is_high_heatwave' (lat: 432, lon: 571)>
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1]])
Coordinates:
  * lon      (lon) float64 -124.8 -124.7 -124.7 -124.6 ... -101.1 -101.1 -101.0
  * lat      (lat) float64 48.98 48.94 48.9 48.86 ... 31.15 31.11 31.07 31.02
    day      datetime64[ns] 2021-12-21
/data2/lthapa/2021/GRIDMET/gridmet_all_2021-12-21.nc

2021-12-22T00:00
<xarray.DataArray 'is_high_heatwave' (lat: 432, lon: 571)>
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1]])
Coordinates:
  * lon      (lon) float64 -124.8 -124.7 -124.7 -124.6 ... -101.1 -101.1 -101.0
  * lat      (lat) float64 48.98 48.94 48.9 48.86 ... 31.15 31.11 31.07 31.02
    day      datetim

# preprocess gridmet, make the files daily with all variables

In [36]:
years = [2019,2020,2021]
#years = [2019]

for jj in range(len(years)):
    gridmet_filenames =os.popen('ls /data2/lthapa/'+str(years[jj])+'/GRIDMET/*_'+str(years[jj])+'.nc').read().split('\n')
    print(gridmet_filenames[0:len(gridmet_filenames)-1])
    dat_gridmet = xr.open_mfdataset(gridmet_filenames[0:len(gridmet_filenames)-1], combine='nested',compat='override', coords='all')

    days = np.arange(np.datetime64(str(years[jj])+'-06-01 00:00'),np.datetime64(str(years[jj]+1)+'-01-01 00:00'),np.timedelta64(1,'D'))

    for kk in range(len(days)):
        today = days[kk]
        print(today)
        #select today and make it western US
        dat_gridmet_today = dat_gridmet.sel(day=today,lon=slice(-125, -101), lat=slice(49,31))
        #print(dat_gridmet_today)
        
        #load in the 90th percentile for this day
        day_string = str(days[kk])[5:10]
        heatwave_threshold_today = xr.open_dataset('/data2/lthapa/TMAX_CLIMO/climo_tmmx_90th_'+day_string+'.nc')
        #print(heatwave_threshold_today)
        is_heatwave = xr.where(dat_gridmet_today['max_air_temperature']>heatwave_threshold_today['air_temperature'],
                              1,0)

        dat_gridmet_today['is_heatwave'] = (('lat','lon'), is_heatwave)
        
        
        #send to netcdf
        file_name_save = '/data2/lthapa/'+str(years[jj])+'/GRIDMET/gridmet_all_'+np.datetime_as_string(today,unit='D')+'.nc'
        print(file_name_save)
        dat_gridmet_today.to_netcdf(file_name_save)
        print()

        

['/data2/lthapa/2019/GRIDMET/bi_2019.nc', '/data2/lthapa/2019/GRIDMET/erc_2019.nc', '/data2/lthapa/2019/GRIDMET/etr_2019.nc', '/data2/lthapa/2019/GRIDMET/fm1000_2019.nc', '/data2/lthapa/2019/GRIDMET/fm100_2019.nc', '/data2/lthapa/2019/GRIDMET/pet_2019.nc', '/data2/lthapa/2019/GRIDMET/pr_2019.nc', '/data2/lthapa/2019/GRIDMET/rmax_2019.nc', '/data2/lthapa/2019/GRIDMET/rmin_2019.nc', '/data2/lthapa/2019/GRIDMET/sph_2019.nc', '/data2/lthapa/2019/GRIDMET/srad_2019.nc', '/data2/lthapa/2019/GRIDMET/th_2019.nc', '/data2/lthapa/2019/GRIDMET/tmmn_2019.nc', '/data2/lthapa/2019/GRIDMET/tmmx_2019.nc', '/data2/lthapa/2019/GRIDMET/vpd_2019.nc', '/data2/lthapa/2019/GRIDMET/vs_2019.nc']
2019-06-01T00:00
/data2/lthapa/2019/GRIDMET/gridmet_all_2019-06-01.nc

2019-06-02T00:00
/data2/lthapa/2019/GRIDMET/gridmet_all_2019-06-02.nc

2019-06-03T00:00
/data2/lthapa/2019/GRIDMET/gridmet_all_2019-06-03.nc

2019-06-04T00:00
/data2/lthapa/2019/GRIDMET/gridmet_all_2019-06-04.nc

2019-06-05T00:00
/data2/lthapa/2019/G


2019-09-15T00:00
/data2/lthapa/2019/GRIDMET/gridmet_all_2019-09-15.nc

2019-09-16T00:00
/data2/lthapa/2019/GRIDMET/gridmet_all_2019-09-16.nc

2019-09-17T00:00
/data2/lthapa/2019/GRIDMET/gridmet_all_2019-09-17.nc

2019-09-18T00:00
/data2/lthapa/2019/GRIDMET/gridmet_all_2019-09-18.nc

2019-09-19T00:00
/data2/lthapa/2019/GRIDMET/gridmet_all_2019-09-19.nc

2019-09-20T00:00
/data2/lthapa/2019/GRIDMET/gridmet_all_2019-09-20.nc

2019-09-21T00:00
/data2/lthapa/2019/GRIDMET/gridmet_all_2019-09-21.nc

2019-09-22T00:00
/data2/lthapa/2019/GRIDMET/gridmet_all_2019-09-22.nc

2019-09-23T00:00
/data2/lthapa/2019/GRIDMET/gridmet_all_2019-09-23.nc

2019-09-24T00:00
/data2/lthapa/2019/GRIDMET/gridmet_all_2019-09-24.nc

2019-09-25T00:00
/data2/lthapa/2019/GRIDMET/gridmet_all_2019-09-25.nc

2019-09-26T00:00
/data2/lthapa/2019/GRIDMET/gridmet_all_2019-09-26.nc

2019-09-27T00:00
/data2/lthapa/2019/GRIDMET/gridmet_all_2019-09-27.nc

2019-09-28T00:00
/data2/lthapa/2019/GRIDMET/gridmet_all_2019-09-28.nc

2019-

/data2/lthapa/2020/GRIDMET/gridmet_all_2020-06-01.nc

2020-06-02T00:00
/data2/lthapa/2020/GRIDMET/gridmet_all_2020-06-02.nc

2020-06-03T00:00
/data2/lthapa/2020/GRIDMET/gridmet_all_2020-06-03.nc

2020-06-04T00:00
/data2/lthapa/2020/GRIDMET/gridmet_all_2020-06-04.nc

2020-06-05T00:00
/data2/lthapa/2020/GRIDMET/gridmet_all_2020-06-05.nc

2020-06-06T00:00
/data2/lthapa/2020/GRIDMET/gridmet_all_2020-06-06.nc

2020-06-07T00:00
/data2/lthapa/2020/GRIDMET/gridmet_all_2020-06-07.nc

2020-06-08T00:00
/data2/lthapa/2020/GRIDMET/gridmet_all_2020-06-08.nc

2020-06-09T00:00
/data2/lthapa/2020/GRIDMET/gridmet_all_2020-06-09.nc

2020-06-10T00:00
/data2/lthapa/2020/GRIDMET/gridmet_all_2020-06-10.nc

2020-06-11T00:00
/data2/lthapa/2020/GRIDMET/gridmet_all_2020-06-11.nc

2020-06-12T00:00
/data2/lthapa/2020/GRIDMET/gridmet_all_2020-06-12.nc

2020-06-13T00:00
/data2/lthapa/2020/GRIDMET/gridmet_all_2020-06-13.nc

2020-06-14T00:00
/data2/lthapa/2020/GRIDMET/gridmet_all_2020-06-14.nc

2020-06-15T00:00
/data2


2020-09-25T00:00
/data2/lthapa/2020/GRIDMET/gridmet_all_2020-09-25.nc

2020-09-26T00:00
/data2/lthapa/2020/GRIDMET/gridmet_all_2020-09-26.nc

2020-09-27T00:00
/data2/lthapa/2020/GRIDMET/gridmet_all_2020-09-27.nc

2020-09-28T00:00
/data2/lthapa/2020/GRIDMET/gridmet_all_2020-09-28.nc

2020-09-29T00:00
/data2/lthapa/2020/GRIDMET/gridmet_all_2020-09-29.nc

2020-09-30T00:00
/data2/lthapa/2020/GRIDMET/gridmet_all_2020-09-30.nc

2020-10-01T00:00
/data2/lthapa/2020/GRIDMET/gridmet_all_2020-10-01.nc

2020-10-02T00:00
/data2/lthapa/2020/GRIDMET/gridmet_all_2020-10-02.nc

2020-10-03T00:00
/data2/lthapa/2020/GRIDMET/gridmet_all_2020-10-03.nc

2020-10-04T00:00
/data2/lthapa/2020/GRIDMET/gridmet_all_2020-10-04.nc

2020-10-05T00:00
/data2/lthapa/2020/GRIDMET/gridmet_all_2020-10-05.nc

2020-10-06T00:00
/data2/lthapa/2020/GRIDMET/gridmet_all_2020-10-06.nc

2020-10-07T00:00
/data2/lthapa/2020/GRIDMET/gridmet_all_2020-10-07.nc

2020-10-08T00:00
/data2/lthapa/2020/GRIDMET/gridmet_all_2020-10-08.nc

2020-


2021-06-09T00:00
/data2/lthapa/2021/GRIDMET/gridmet_all_2021-06-09.nc

2021-06-10T00:00
/data2/lthapa/2021/GRIDMET/gridmet_all_2021-06-10.nc

2021-06-11T00:00
/data2/lthapa/2021/GRIDMET/gridmet_all_2021-06-11.nc

2021-06-12T00:00
/data2/lthapa/2021/GRIDMET/gridmet_all_2021-06-12.nc

2021-06-13T00:00
/data2/lthapa/2021/GRIDMET/gridmet_all_2021-06-13.nc

2021-06-14T00:00
/data2/lthapa/2021/GRIDMET/gridmet_all_2021-06-14.nc

2021-06-15T00:00
/data2/lthapa/2021/GRIDMET/gridmet_all_2021-06-15.nc

2021-06-16T00:00
/data2/lthapa/2021/GRIDMET/gridmet_all_2021-06-16.nc

2021-06-17T00:00
/data2/lthapa/2021/GRIDMET/gridmet_all_2021-06-17.nc

2021-06-18T00:00
/data2/lthapa/2021/GRIDMET/gridmet_all_2021-06-18.nc

2021-06-19T00:00
/data2/lthapa/2021/GRIDMET/gridmet_all_2021-06-19.nc

2021-06-20T00:00
/data2/lthapa/2021/GRIDMET/gridmet_all_2021-06-20.nc

2021-06-21T00:00
/data2/lthapa/2021/GRIDMET/gridmet_all_2021-06-21.nc

2021-06-22T00:00
/data2/lthapa/2021/GRIDMET/gridmet_all_2021-06-22.nc

2021-


2021-10-03T00:00
/data2/lthapa/2021/GRIDMET/gridmet_all_2021-10-03.nc

2021-10-04T00:00
/data2/lthapa/2021/GRIDMET/gridmet_all_2021-10-04.nc

2021-10-05T00:00
/data2/lthapa/2021/GRIDMET/gridmet_all_2021-10-05.nc

2021-10-06T00:00
/data2/lthapa/2021/GRIDMET/gridmet_all_2021-10-06.nc

2021-10-07T00:00
/data2/lthapa/2021/GRIDMET/gridmet_all_2021-10-07.nc

2021-10-08T00:00
/data2/lthapa/2021/GRIDMET/gridmet_all_2021-10-08.nc

2021-10-09T00:00
/data2/lthapa/2021/GRIDMET/gridmet_all_2021-10-09.nc

2021-10-10T00:00
/data2/lthapa/2021/GRIDMET/gridmet_all_2021-10-10.nc

2021-10-11T00:00
/data2/lthapa/2021/GRIDMET/gridmet_all_2021-10-11.nc

2021-10-12T00:00
/data2/lthapa/2021/GRIDMET/gridmet_all_2021-10-12.nc

2021-10-13T00:00
/data2/lthapa/2021/GRIDMET/gridmet_all_2021-10-13.nc

2021-10-14T00:00
/data2/lthapa/2021/GRIDMET/gridmet_all_2021-10-14.nc

2021-10-15T00:00
/data2/lthapa/2021/GRIDMET/gridmet_all_2021-10-15.nc

2021-10-16T00:00
/data2/lthapa/2021/GRIDMET/gridmet_all_2021-10-16.nc

2021-

In [ ]:
dat_gridmet.close()

In [6]:
gridmet_filenames =os.popen('ls /data2/lthapa/2019/GRIDMET/*_2019.nc').read().split('\n')
print(gridmet_filenames[0:len(gridmet_filenames)-1])
dat_gridmet = xr.open_mfdataset(gridmet_filenames[0:len(gridmet_filenames)-1], combine='nested',compat='override', coords='all')

dat_gridmet

['/data2/lthapa/2019/GRIDMET/bi_2019.nc', '/data2/lthapa/2019/GRIDMET/erc_2019.nc', '/data2/lthapa/2019/GRIDMET/etr_2019.nc', '/data2/lthapa/2019/GRIDMET/fm1000_2019.nc', '/data2/lthapa/2019/GRIDMET/fm100_2019.nc', '/data2/lthapa/2019/GRIDMET/pet_2019.nc', '/data2/lthapa/2019/GRIDMET/pr_2019.nc', '/data2/lthapa/2019/GRIDMET/rmax_2019.nc', '/data2/lthapa/2019/GRIDMET/rmin_2019.nc', '/data2/lthapa/2019/GRIDMET/sph_2019.nc', '/data2/lthapa/2019/GRIDMET/srad_2019.nc', '/data2/lthapa/2019/GRIDMET/th_2019.nc', '/data2/lthapa/2019/GRIDMET/tmmn_2019.nc', '/data2/lthapa/2019/GRIDMET/tmmx_2019.nc', '/data2/lthapa/2019/GRIDMET/vpd_2019.nc', '/data2/lthapa/2019/GRIDMET/vs_2019.nc']


<xarray.Dataset>
Dimensions:                                    (crs: 1, day: 365, lat: 585, lon: 1386)
Coordinates:
  * lon                                        (lon) float64 -124.8 ... -67.06
  * lat                                        (lat) float64 49.4 ... 25.07
  * day                                        (day) datetime64[ns] 2019-01-0...
  * crs                                        (crs) uint16 3
Data variables: (12/16)
    burning_index_g                            (day, lat, lon) float32 dask.array<chunksize=(365, 585, 1386), meta=np.ndarray>
    energy_release_component-g                 (day, lat, lon) float32 dask.array<chunksize=(365, 585, 1386), meta=np.ndarray>
    potential_evapotranspiration_alfalfa       (day, lat, lon) float32 dask.array<chunksize=(365, 585, 1386), meta=np.ndarray>
    dead_fuel_moisture_1000hr                  (day, lat, lon) float32 dask.array<chunksize=(365, 585, 1386), meta=np.ndarray>
    dead_fuel_moisture_100hr                   (day, lat, lon) float32 dask.array<chunksize=(365, 585, 1386), meta=np.ndarray>
    potential_evapotranspiration_grass         (day, lat, lon) float32 dask.array<chunksize=(365, 585, 1386), meta=np.ndarray>
    ...                                         ...
    surface_downwelling_shortwave_flux_in_air  (day, lat, lon) float32 dask.array<chunksize=(365, 585, 1386), meta=np.ndarray>
    wind_from_direction                        (day, lat, lon) float32 dask.array<chunksize=(365, 585, 1386), meta=np.ndarray>
    min_air_temperature                        (day, lat, lon) float32 dask.array<chunksize=(365, 585, 1386), meta=np.ndarray>
    max_air_temperature                        (day, lat, lon) float32 dask.array<chunksize=(365, 585, 1386), meta=np.ndarray>
    mean_vapor_pressure_deficit                (day, lat, lon) float32 dask.array<chunksize=(365, 585, 1386), meta=np.ndarray>
    wind_speed                                 (day, lat, lon) float32 dask.array<chunksize=(365, 585, 1386), meta=np.ndarray>
Attributes: (12/19)
    geospatial_bounds_crs:      EPSG:4326
    Conventions:                CF-1.6
    geospatial_bounds:          POLYGON((-124.7666666333333 49.40000000000000...
    geospatial_lat_min:         25.066666666666666
    geospatial_lat_max:         49.40000000000000
    geospatial_lon_min:         -124.7666666333333
    ...                         ...
    date:                       05 May 2021
    note1:                      The projection information for this file is: ...
    note2:                      Citation: Abatzoglou, J.T., 2013, Development...
    note3:                      Data in slices after last_permanent_slice (1-...
    note4:                      Data in slices after last_provisional_slice (...
    note5:                      Days correspond approximately to calendar day...

In [5]:
days = np.arange(np.datetime64('2019-06-01 00:00'),np.datetime64('2020-01-01 00:00'),np.timedelta64(1,'D'))

dat_gridmet_today = dat_gridmet.sel(day=days[0])
dat_gridmet_today


<xarray.Dataset>
Dimensions:                                    (crs: 1, lat: 585, lon: 1386)
Coordinates:
  * lon                                        (lon) float64 -124.8 ... -67.06
  * lat                                        (lat) float64 49.4 ... 25.07
    day                                        datetime64[ns] 2019-06-01
  * crs                                        (crs) uint16 3
Data variables: (12/16)
    burning_index_g                            (lat, lon) float32 dask.array<chunksize=(585, 1386), meta=np.ndarray>
    energy_release_component-g                 (lat, lon) float32 dask.array<chunksize=(585, 1386), meta=np.ndarray>
    potential_evapotranspiration_alfalfa       (lat, lon) float32 dask.array<chunksize=(585, 1386), meta=np.ndarray>
    dead_fuel_moisture_1000hr                  (lat, lon) float32 dask.array<chunksize=(585, 1386), meta=np.ndarray>
    dead_fuel_moisture_100hr                   (lat, lon) float32 dask.array<chunksize=(585, 1386), meta=np.ndarray>
    potential_evapotranspiration_grass         (lat, lon) float32 dask.array<chunksize=(585, 1386), meta=np.ndarray>
    ...                                         ...
    surface_downwelling_shortwave_flux_in_air  (lat, lon) float32 dask.array<chunksize=(585, 1386), meta=np.ndarray>
    wind_from_direction                        (lat, lon) float32 dask.array<chunksize=(585, 1386), meta=np.ndarray>
    min_air_temperature                        (lat, lon) float32 dask.array<chunksize=(585, 1386), meta=np.ndarray>
    max_air_temperature                        (lat, lon) float32 dask.array<chunksize=(585, 1386), meta=np.ndarray>
    mean_vapor_pressure_deficit                (lat, lon) float32 dask.array<chunksize=(585, 1386), meta=np.ndarray>
    wind_speed                                 (lat, lon) float32 dask.array<chunksize=(585, 1386), meta=np.ndarray>
Attributes: (12/19)
    geospatial_bounds_crs:      EPSG:4326
    Conventions:                CF-1.6
    geospatial_bounds:          POLYGON((-124.7666666333333 49.40000000000000...
    geospatial_lat_min:         25.066666666666666
    geospatial_lat_max:         49.40000000000000
    geospatial_lon_min:         -124.7666666333333
    ...                         ...
    date:                       05 May 2021
    note1:                      The projection information for this file is: ...
    note2:                      Citation: Abatzoglou, J.T., 2013, Development...
    note3:                      Data in slices after last_permanent_slice (1-...
    note4:                      Data in slices after last_provisional_slice (...
    note5:                      Days correspond approximately to calendar day...

In [ ]:
print(dat_gridmet['day'])